### SQL Questions for Technical Test

#### Question 1

Of the industries where salaries and wages data did not exist in 2016, 'Retail Trade' was the industry with the highest average value for actual filled jobs across time with and average number of Actual filled jobs of 194,000 (rounded to the nearest thousand.)

#### Question 2

The year/month combination and industry that had the second highest seasonally adjusted operating income sales across all the business industries categorised as NZSIOC level 2 was `2023.03` and `Wholesale Trade` which had Seasonally adjusted Sales and Operating Income of NZD $38,810,022,000

#### Question 3

SQL Query below

In [1]:
import duckdb
import pandas as pd

In [2]:
# All code used to create these tidied tables found in notebook.ipynb
salaries_and_wages_df = pd.read_excel('data/salaries_and_wages_df.xlsx')
filled_jobs_by_industry_df = pd.read_excel('data/filled_jobs_by_industry_df.xlsx')
filled_jobs_by_territorial_authority_df = pd.read_excel('data/filled_jobs_by_territorial_authority_df.xlsx')
sales_and_operating_income_df = pd.read_excel('data/sales_and_operating_income_df.xlsx')

# salaries_and_wages table
duckdb.sql("""
        CREATE TABLE salaries_and_wages
        AS SELECT *
        FROM salaries_and_wages_df
        """)

# employment_by_industry table
duckdb.sql("""
        CREATE TABLE employment_by_industry
        AS SELECT *
        FROM filled_jobs_by_industry_df
        """)

# employment_by_territorial_authority table
duckdb.sql("""
        CREATE TABLE employment_by_territorial_authority
        AS SELECT *
        FROM filled_jobs_by_territorial_authority_df
        """)

# sales and operating income table
duckdb.sql("""
        CREATE TABLE sales_and_operating_income
        AS SELECT *
        FROM sales_and_operating_income_df
        """)

In [3]:
# Question 1
duckdb.sql("""
    WITH industries_of_interest AS (
        SELECT industry_type, MIN(period) as earliest_period
        FROM salaries_and_wages
        GROUP BY industry_type
        HAVING earliest_period > 2016.13
    )
    SELECT industry_type, AVG(data_value) as avg_filled_jobs
    FROM employment_by_industry
    WHERE Adjustment = 'Actual'
    GROUP BY industry_type
    HAVING industry_type IN (SELECT industry_type FROM industries_of_interest)
    ORDER BY avg_filled_jobs DESC
    LIMIT 1;
""")

┌───────────────┬────────────────────┐
│ Industry_Type │  avg_filled_jobs   │
│    varchar    │       double       │
├───────────────┼────────────────────┤
│ Retail Trade  │ 194053.71153846153 │
└───────────────┴────────────────────┘

In [4]:
# Question 2
duckdb.sql("""
        SELECT Period, Industry_Type, Data_value as sales_and_operating_income
        FROM sales_and_operating_income
        WHERE Adjustment = 'Seasonally adjusted'
        AND NZIOC_Level = 2
        ORDER BY Data_value DESC
        LIMIT 1 OFFSET 1
        """)

┌─────────┬─────────────────┬────────────────────────────┐
│ Period  │  Industry_Type  │ sales_and_operating_income │
│ double  │     varchar     │           double           │
├─────────┼─────────────────┼────────────────────────────┤
│ 2023.03 │ Wholesale Trade │                  38810.022 │
└─────────┴─────────────────┴────────────────────────────┘

In [5]:
# Question 3
duckdb.sql("""
    SELECT a.Territorial_Authority, a.Period, SUM(b.Data_value) as cumulative_filled_jobs
    FROM employment_by_territorial_authority a
    JOIN employment_by_territorial_authority b
    ON a.Period >= b.Period
    WHERE a.Territorial_Authority = (
        SELECT Territorial_Authority
        FROM employment_by_territorial_authority
        GROUP BY Territorial_Authority
        ORDER BY AVG(Data_value) DESC
        LIMIT 1
    )
    GROUP BY a.Territorial_Authority, a.Period
    ORDER BY a.Period;
""")

┌───────────────────────┬─────────┬────────────────────────┐
│ Territorial_Authority │ Period  │ cumulative_filled_jobs │
│        varchar        │ double  │         double         │
├───────────────────────┼─────────┼────────────────────────┤
│ Auckland              │ 2011.06 │              1726876.0 │
│ Auckland              │ 2011.09 │              3451994.0 │
│ Auckland              │ 2011.12 │              5207022.0 │
│ Auckland              │ 2012.03 │              6935935.0 │
│ Auckland              │ 2012.06 │              8693257.0 │
│ Auckland              │ 2012.09 │             10430964.0 │
│ Auckland              │ 2012.12 │             12200120.0 │
│ Auckland              │ 2013.03 │             13957409.0 │
│ Auckland              │ 2013.06 │             15745761.0 │
│ Auckland              │ 2013.09 │             17518802.0 │
│    ·                  │    ·    │                  ·     │
│    ·                  │    ·    │                  ·     │
│    ·                  